# Playing with model editing

In [1]:
from pathlib import Path
if Path().resolve().name == "notebooks":
    %cd ..
%ls
import sys
sys.path.append("models")
sys.path.append("data")
sys.path.append("concepts")

/Users/dgcnz/development/uva/fact/FACT
LICENSE                               notebook/
README.md                             notebooks/
artifacts/                            poetry.lock
assets/                               pyproject.toml
concepts/                             scripts/
configs/                              test_cav_activation.py
data/                                 test_different_projections.py
docs/                                 testnotebook.ipynb
environment.yaml                      train_pcbm.py
experiments/                          train_pcbm_h.py
extension_scripts/                    training_tools/
learn_concepts_dataset.py             verify_clip_pcbm_h.py
learn_concepts_multimodal.py          verify_dataset_pcbm.py
lightning_logs/                       verify_dataset_pcbm_h.py
models/                               verify_results_clip_concepts_pcbm.py


/Users/dgcnz/development/uva/fact/FACT/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from models import PosthocLinearCBM, get_model
from collections import namedtuple
from data import get_dataset
from concepts import ConceptBank
import pickle
import torch

In [3]:
BACKBONE_NAME = "clip:RN50"
DEVICE = "cpu"
CONCEPT_BANK = "artifacts/outdir/broden_clip_RN50_10.0_50.pkl"
PCBM_PATH = "artifacts/outdir/pcbm_cifar10__clip_RN50__broden_clip_RN50_10__lam_0.0002__alpha_0.99__seed_42.ckpt"
args = namedtuple(
    "Args",
    [
        "out_dir",
        "device",
        "dataset",
        "batch_size",
        "num_workers",
    ],
)(
    out_dir="artifacts/outdir",
    device=DEVICE,
    dataset="cifar10",
    batch_size=64,
    num_workers=4,
)
all_concepts = pickle.load(open(CONCEPT_BANK, 'rb'))
all_concept_names = list(all_concepts.keys())
print(f"Bank path: {CONCEPT_BANK}. {len(all_concept_names)} concepts will be used.")
concept_bank = ConceptBank(all_concepts, args.device)
backbone, preprocess = get_model(args, backbone_name=BACKBONE_NAME)
train_loader, test_loader, idx_to_class, classes = get_dataset(args, preprocess)
num_classes = len(classes)
backbone = backbone.to(DEVICE)
backbone.eval()
# posthoc_layer = PosthocLinearCBM(
#     concept_bank,
#     backbone_name=BACKBONE_NAME,
#     idx_to_class=idx_to_class,
#     n_classes=num_classes,
# )
# posthoc_layer = posthoc_layer.to(args.device)
posthoc_layer = torch.load(PCBM_PATH, map_location=torch.device('cpu'))

Bank path: artifacts/outdir/broden_clip_RN50_10.0_50.pkl. 170 concepts will be used.
Concept Bank is initialized.
Files already downloaded and verified
Files already downloaded and verified


In [4]:
print(len(concept_bank.concept_names))

170


In [11]:
d = dict(zip(concept_bank.concept_names, list(range(len(concept_bank.concept_names)))))
d["dog"]

86

In [13]:
d["book"]

7

In [15]:
list(d.keys())[:10]

['door_frame',
 'concrete',
 'horse',
 'food',
 'oven',
 'mouse',
 'bedroom_s',
 'book',
 'chain_wheel',
 'blueness']

In [11]:
idx_to_class

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [10]:
all_concepts["door_frame"][0].shape

(1, 1024)

In [ ]:
all_concepts[""]

In [47]:
posthoc_layer.eval()

PosthocLinearCBM(
  (classifier): Linear(in_features=170, out_features=10, bias=True)
)

In [53]:
posthoc_layer.classifier.weight.shape

torch.Size([10, 170])

In [49]:
posthoc_layer.analyze_classifier().split("\n")

['Class : airplane',
 '\t 1 - airplane: 13.058',
 '\t 2 - body: 5.333',
 '\t 3 - bench: 5.237',
 '\t 4 - fluorescent: 5.024',
 '\t 5 - building: 4.217',
 'Class : automobile',
 '\t 1 - headlight: 13.475',
 '\t 2 - car: 7.165',
 '\t 3 - door: 4.991',
 '\t 4 - flower: 3.321',
 '\t 5 - motorbike: 3.272',
 'Class : bird',
 '\t 1 - bird: 9.885',
 '\t 2 - house: 7.090',
 '\t 3 - foot: 6.455',
 '\t 4 - ashcan: 4.173',
 '\t 5 - handle_bar: 3.579',
 'Class : cat',
 '\t 1 - cat: 9.937',
 '\t 2 - foot: 6.573',
 '\t 3 - floor: 6.013',
 '\t 4 - street_s: 4.118',
 '\t 5 - ceramic: 3.946',
 'Class : deer',
 '\t 1 - street_s: 8.018',
 '\t 2 - tree: 6.953',
 '\t 3 - engine: 6.178',
 '\t 4 - clock: 5.866',
 '\t 5 - pipe: 5.183',
 'Class : dog',
 '\t 1 - paw: 8.105',
 '\t 2 - muzzle: 6.343',
 '\t 3 - board: 6.019',
 '\t 4 - exhaust_hood: 5.522',
 '\t 5 - keyboard: 5.193',
 'Class : frog',
 '\t 1 - hand: 6.303',
 '\t 2 - greenness: 6.221',
 '\t 3 - food: 4.601',
 '\t 4 - carpet: 4.462',
 '\t 5 - pipe: 4.1